# ML Project: Regression of Zillow Home Value Index for Virginia Housing
## Nanda, Siddharth (sn9dq); Wilson, Dale (dsw6ru)

In [ ]:
# Import some common packages
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

from pandas.plotting import scatter_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

# to make this notebook's output stable across runs
np.random.seed(42)

In [17]:
# Read in data/split
housing = pd.read_csv('Zip_Zhvi_Summary_AllHomes.csv', encoding='latin-1')
housing = housing.drop(columns=['County', 'City', 'PeakMonth', 'Metro', 'PeakQuarter', 'LastTimeAtCurrZHVI'])
va = housing.loc[housing['State'] == 'VA'].dropna()
housing = housing.dropna()

train_set, test_set = train_test_split(housing, test_size = 0.2)
print("Description of label")
display(housing['Zhvi'].describe())
y_train = train_set['Zhvi']
X_train = train_set.drop(columns=['Zhvi'])

y_test = test_set['Zhvi']
X_test = test_set.drop(columns=['Zhvi'])

Description of label


count    1.496800e+04
mean     2.816919e+05
std      2.734603e+05
min      2.430000e+04
25%      1.395000e+05
50%      2.061000e+05
75%      3.246500e+05
max      6.493800e+06
Name: Zhvi, dtype: float64

In [18]:
# Pipelines for data transformation
housing = housing.drop(columns=['Zhvi'])
num_pipeline = Pipeline([
    ('standard_scaler', StandardScaler())
])

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, list(housing.select_dtypes(include=[np.number]))),
    ('cat', OneHotEncoder(), list(housing.select_dtypes(include=[np.object]).columns))
])

In [19]:
# fit_transform train set, and transform test set (scaling!)
X_train = full_pipeline.fit_transform(X_train)
X_test = full_pipeline.transform(X_test)

va_y = va['Zhvi']
va_x = va.drop(columns=['Zhvi'])
va_x = full_pipeline.transform(va_x)

/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred = lin_reg.predict(X_test)
print("National LinReg")
print(r2_score(y_test, y_pred))
print(np.sqrt(mean_squared_error(y_test, y_pred)))

National LinReg
0.9954922048847467
19205.981491861116


In [21]:
print("Virginia LinReg")
y_pred = lin_reg.predict(va_x)
print(r2_score(va_y, y_pred))
print(np.sqrt(mean_squared_error(va_y, y_pred)))

Virginia LinReg
0.9913687068461461
15814.483851067685
